In [1]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
# import glove nitin commented as its not used
# from glove import Corpus

import collections
import gc 

import keras
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Input, concatenate, merge, Activation, Concatenate, LSTM, GRU
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, BatchNormalization, GRU, Convolution1D, LSTM
from keras.layers import UpSampling1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D,MaxPool1D, merge

# from keras.optimizers import Adam # nitin commented as Adam has been shifted to optimizer_v1 module.
from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping, ModelCheckpoint, History, ReduceLROnPlateau
from keras.utils import np_utils
#from keras.backend.tensorflow_backend import set_session, clear_session, get_session # nitin commented as tensorflow_backend not used anymore
from keras.backend import set_session, clear_session, get_session
import tensorflow as tf


from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
# nitin added as without it the read_pickel below was giving the error as in notebook 5 when its writing
#pickel do a lazy execution and write the function as well
def mean(a):
    return sum(a) / len(a)

In [3]:
type_of_ner = "new"

x_train_lstm = pd.read_pickle("data/"+type_of_ner+"_x_train.pkl")
x_dev_lstm = pd.read_pickle("data/"+type_of_ner+"_x_dev.pkl")
x_test_lstm = pd.read_pickle("data/"+type_of_ner+"_x_test.pkl")

y_train = pd.read_pickle("data/"+type_of_ner+"_y_train.pkl")
y_dev = pd.read_pickle("data/"+type_of_ner+"_y_dev.pkl")
y_test = pd.read_pickle("data/"+type_of_ner+"_y_test.pkl")


ner_word2vec = pd.read_pickle("data/"+type_of_ner+"_ner_word2vec_limited_dict.pkl")
ner_fasttext = pd.read_pickle("data/"+type_of_ner+"_ner_fasttext_limited_dict.pkl")
ner_concat = pd.read_pickle("data/"+type_of_ner+"_ner_combined_limited_dict.pkl")

train_ids = pd.read_pickle("data/"+type_of_ner+"_train_ids.pkl")
dev_ids = pd.read_pickle("data/"+type_of_ner+"_dev_ids.pkl")
test_ids = pd.read_pickle("data/"+type_of_ner+"_test_ids.pkl")

In [4]:
x_train_lstm.shape,x_dev_lstm.shape,x_test_lstm.shape ,y_train.shape,y_dev.shape,y_test.shape

((15219, 24, 104),
 (2164, 24, 104),
 (4348, 24, 104),
 (15219, 4),
 (2164, 4),
 (4348, 4))

In [5]:
def make_prediction_cnn(model, test_data):
    probs = model.predict(test_data)
    y_pred = [1 if i>=0.5 else 0 for i in probs]
    return probs, y_pred

def save_scores_cnn(predictions, probs, ground_truth, 
                          
                          embed_name, problem_type, iteration, hidden_unit_size,
                          
                          sequence_name, type_of_ner):
    
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    result_dict = {}    
    result_dict['auc'] = auc
    result_dict['auprc'] = auprc
    result_dict['acc'] = acc
    result_dict['F1'] = F1

    result_path = "results/cnn/"
    file_name = str(sequence_name)+"-"+str(hidden_unit_size)+"-"+embed_name
    file_name = file_name +"-"+problem_type+"-"+str(iteration)+"-"+type_of_ner+"-cnn-.p"
    pd.to_pickle(result_dict, os.path.join(result_path, file_name))

    print(auc, auprc, acc, F1)
    
def print_scores_cnn(predictions, probs, ground_truth, model_name, problem_type, iteration, hidden_unit_size):
    auc = roc_auc_score(ground_truth, probs)
    auprc = average_precision_score(ground_truth, probs)
    acc   = accuracy_score(ground_truth, predictions)
    F1    = f1_score(ground_truth, predictions)
    
    print ("AUC: ", auc, "AUPRC: ", auprc, "F1: ", F1)
    
def get_subvector_data(size, embed_name, data):
    if embed_name == "concat":
        vector_size = 200
    else:
        vector_size = 100

    x_data = {}
    for k, v in data.items():
        number_of_additional_vector = len(v) - size
        vector = []
        for i in v:
            vector.append(i)
        if number_of_additional_vector < 0: 
            number_of_additional_vector = np.abs(number_of_additional_vector)

            temp = vector[:size]
            for i in range(0, number_of_additional_vector):
                temp.append(np.zeros(vector_size))
            x_data[k] = np.asarray(temp)
        else:
            x_data[k] = np.asarray(vector[:size])

    return x_data


def proposedmodel(layer_name, number_of_unit, embedding_name, ner_limit, num_filter):
    if embedding_name == "concat":
        input_dimension = 200
    else:
        input_dimension = 100

    sequence_input = Input(shape=(24,104))

    input_img = Input(shape=(ner_limit, input_dimension), name = "cnn_input")

    convs = []
    filter_sizes = [2,3,4]


    #logits_regularizer = tf.compat.v1.estimator.layers.l2_regularizer(scale=0.01)
    # removed tf.compat.v1.estimator.layers.xavier_initializer() with glorot_normal
    
    text_conv1d = Conv1D(filters=num_filter, kernel_size=3, 
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu', 
                         kernel_initializer=tf.keras.initializers.glorot_normal() )(input_img)
    
    text_conv1d = Conv1D(filters=num_filter*2, kernel_size=3, 
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=tf.keras.initializers.glorot_normal())(text_conv1d)   
    
    text_conv1d = Conv1D(filters=num_filter*3, kernel_size=3, 
                 padding = 'valid', strides = 1, dilation_rate=1, activation='relu',
                        kernel_initializer=tf.keras.initializers.glorot_normal())(text_conv1d)   

    
    #concat_conv = keras.layers.Concatenate()([text_conv1d, text_conv1d_2, text_conv1d_3])
    text_embeddings = GlobalMaxPooling1D()(text_conv1d)
    #text_embeddings = Dense(128, activation="relu")(text_embeddings)
    
    if layer_name == "GRU":
        x = GRU(number_of_unit)(sequence_input)
    elif layer_name == "LSTM":
        x = LSTM(number_of_unit)(sequence_input)

    concatenated = keras.layers.Concatenate(axis =1)([x, text_embeddings])# nitin uncommented
    #concatenated = merge([x, text_embeddings], mode='concat', concat_axis=1)# nitin commented

    concatenated = Dense(512, activation='relu')(concatenated)
    concatenated = Dropout(0.2)(concatenated)
    #concatenated = Dense(256, activation='relu')(concatenated)
    #concatenated = Dense(512, activation='relu')(concatenated)
    
    #concatenated = Dense(512, activation='relu')(concatenated)
    
    #logits_regularizer = tf.contrib.layers.l2_regularizer(scale=0.01) nitin commented as its deprecated
    logits_regularizer = tf.keras.regularizers.L2(0.01)
    
    preds = Dense(1, activation='sigmoid',use_bias=False,
                         kernel_initializer=tf.keras.initializers.glorot_normal(), 
                  kernel_regularizer=logits_regularizer)(concatenated)
    
    
    #opt = Adam(lr=1e-4, decay = 0.01)
    
    opt = Adam(lr=1e-3, decay = 0.01)
    
    #opt = Adam(lr=0.001)

    model = Model(inputs=[sequence_input, input_img], outputs=preds)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['acc'])
    
    return model


dict_f = {"a": [1,2,3], "b": [1,2,3], "c": [1,2,3], "d": [1,2,3]}

#data = list(dict.items())
data = collections.OrderedDict(sorted(dict_f.items()))
print(sorted(dict_f.items()))
an_array = np.asarray(data.values())

print(data)
print('an_array',an_array)
print(type(data))



for a in data.items():
    print(a[1])

t1[0]

i =0
t = []
for kk,v in x_train_dict_sorted.items():
    i+=1
    #print(kk, v.shape,i+1)
    t.append(v)
a  = np.array(t)
    

a.shape

print(a)

In [6]:
embedding_types = ['word2vec', 'fasttext']#, 'concat']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]

target_problems = ['mort_hosp', 'mort_icu']#, 'los_3', 'los_7']

num_epoch = 100
model_patience = 5
monitor_criteria = 'val_loss'
#monitor_criteria = 'val_acc'
batch_size = 64

filter_number = 32
ner_representation_limit = 64
activation_func = "relu"

sequence_model = "GRU"
sequence_hidden_unit = 256

maxiter = 11
for embed_dict, embed_name in zip(embedding_dict, embedding_types):    
    print ("Embedding: ", embed_name)
    print("=============================")
    
    temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
    tem_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
    temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

    x_train_dict = {}
    x_dev_dict = {}
    x_test_dict = {}

    x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
    x_dev_dict = get_subvector_data(ner_representation_limit, embed_name, tem_dev_ner)
    x_test_dict = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

    x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
    x_dev_dict_sorted = collections.OrderedDict(sorted(x_dev_dict.items()))
    x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))
    
#     x_train_dict_sorted = sorted(x_train_dict.items())
#     x_dev_dict_sorted = sorted(x_dev_dict.items())
#     x_test_dict_sorted = sorted(x_test_dict.items())
    list_train_ner =[]
    list_test_ner =[]
    list_dev_ner =[]
    for k,v in x_train_dict_sorted.items():
            list_train_ner.append(v)
    for k,v in x_test_dict_sorted.items():
            list_test_ner.append(v)
    for k,v in x_dev_dict_sorted.items():
            list_dev_ner.append(v)
    
        
    
    x_train_ner = np.array(list_train_ner)
    x_dev_ner = np.array(list_dev_ner)
    x_test_ner = np.array(list_test_ner)
    
    print(x_train_ner.shape)
    

    #print('shape of x_train_ner{0} x_dev_ner{1} x_test_ner {2}'.format(x_train_ner.shape,x_dev_ner.shape,x_test_ner.shape))
    
        
    for iteration in range(1,maxiter):
        print ("Iteration number: ", iteration)
    
        for each_problem in target_problems:
            print ("Problem type: ", each_problem)
            print ("__________________")
            
            
            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
            
            best_model_name = str(ner_representation_limit)+"-basiccnn1d-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
            
            checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
                save_best_only=True, mode='min')
            
            reduce_lr = ReduceLROnPlateau(monitor=monitor_criteria, factor=0.2,
                              patience=2, min_lr=0.00001, epsilon=1e-4, mode='min')
            

            callbacks = [early_stopping_monitor, checkpoint, reduce_lr]
            
            #model = textCNN(sequence_model, sequence_hidden_unit, embed_name, ner_representation_limit)
            model = proposedmodel(sequence_model, sequence_hidden_unit, 
                               embed_name, ner_representation_limit,filter_number)
            print('x_train_lstm {0} x_dev_lstm {1}'.format(x_train_lstm.shape,x_dev_lstm.shape))
            print('x_train_ner {0} x_dev_ner {1}'.format(x_train_ner.shape,x_dev_ner.shape))
            print(y_train[each_problem].shape)
            aa = [x_train_lstm, x_train_ner]
            print(x_train_lstm.shape, x_train_ner.shape, len(aa))
            model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                      validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, batch_size=batch_size)
            
            
            probs, predictions = make_prediction_cnn(model, [x_test_lstm, x_test_ner])
            print_scores_cnn(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration, sequence_hidden_unit)
            
            model.load_weights(best_model_name)
                      
            probs, predictions = make_prediction_cnn(model, [x_test_lstm, x_test_ner])
            save_scores_cnn(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration,
                            sequence_hidden_unit, sequence_model, type_of_ner)
            del model
            clear_session()
            gc.collect()
            
    

Embedding:  word2vec
(15219, 64, 100)
Iteration number:  1
Problem type:  mort_hosp
__________________
x_train_lstm (15219, 24, 104) x_dev_lstm (2164, 24, 104)
x_train_ner (15219, 64, 100) x_dev_ner (2164, 64, 100)
(15219,)
(15219, 24, 104) (15219, 64, 100) 2
Epoch 1/100
238/238 [==============================] - ETA: 0s - loss: 0.2879 - acc: 0.9004
Epoch 00001: val_loss improved from inf to 0.24888, saving model to 64-basiccnn1d-word2vec-mort_hosp-best_model.hdf5
238/238 [==============================] - 30s 92ms/step - loss: 0.2879 - acc: 0.9004 - val_loss: 0.2489 - val_acc: 0.9145 - lr: 0.0010
Epoch 2/100
238/238 [==============================] - ETA: 0s - loss: 0.2363 - acc: 0.9164
Epoch 00002: val_loss improved from 0.24888 to 0.23846, saving model to 64-basiccnn1d-word2vec-mort_hosp-best_model.hdf5
238/238 [==============================] - 21s 88ms/step - loss: 0.2363 - acc: 0.9164 - val_loss: 0.2385 - val_acc: 0.9159 - lr: 0.0010
Epoch 3/100
238/238 [=========================

AUC:  0.8836307163626752 AUPRC:  0.5922632618159211 F1:  0.48137535816618904
0.8824147501982554 0.5895581760838886 0.9169733210671573 0.46518518518518515
Problem type:  mort_icu
__________________
x_train_lstm (15219, 24, 104) x_dev_lstm (2164, 24, 104)
x_train_ner (15219, 64, 100) x_dev_ner (2164, 64, 100)
(15219,)
(15219, 24, 104) (15219, 64, 100) 2
Epoch 1/100
238/238 [==============================] - ETA: 0s - loss: 0.2162 - acc: 0.9346
Epoch 00001: val_loss improved from inf to 0.17858, saving model to 64-basiccnn1d-word2vec-mort_icu-best_model.hdf5
238/238 [==============================] - 27s 88ms/step - loss: 0.2162 - acc: 0.9346 - val_loss: 0.1786 - val_acc: 0.9427 - lr: 0.0010
Epoch 2/100
238/238 [==============================] - ETA: 0s - loss: 0.1743 - acc: 0.9430
Epoch 00002: val_loss improved from 0.17858 to 0.17593, saving model to 64-basiccnn1d-word2vec-mort_icu-best_model.hdf5
238/238 [==============================] - 20s 84ms/step - loss: 0.1743 - acc: 0.9430 - va

Epoch 5/100
238/238 [==============================] - ETA: 0s - loss: 0.1363 - acc: 0.9533
Epoch 00005: val_loss did not improve from 0.16896
238/238 [==============================] - 19s 82ms/step - loss: 0.1363 - acc: 0.9533 - val_loss: 0.1691 - val_acc: 0.9413 - lr: 0.0010
Epoch 6/100
238/238 [==============================] - ETA: 0s - loss: 0.1268 - acc: 0.9547
Epoch 00006: val_loss did not improve from 0.16896
238/238 [==============================] - 21s 86ms/step - loss: 0.1268 - acc: 0.9547 - val_loss: 0.1732 - val_acc: 0.9422 - lr: 0.0010
Epoch 7/100
238/238 [==============================] - ETA: 0s - loss: 0.1177 - acc: 0.9592
Epoch 00007: val_loss did not improve from 0.16896
238/238 [==============================] - 20s 85ms/step - loss: 0.1177 - acc: 0.9592 - val_loss: 0.1728 - val_acc: 0.9418 - lr: 2.0000e-04
Epoch 8/100
238/238 [==============================] - ETA: 0s - loss: 0.1155 - acc: 0.9598
Epoch 00008: val_loss did not improve from 0.16896
238/238 [=======

Epoch 3/100
238/238 [==============================] - ETA: 0s - loss: 0.2239 - acc: 0.9206
Epoch 00003: val_loss did not improve from 0.23918
238/238 [==============================] - 17s 70ms/step - loss: 0.2239 - acc: 0.9206 - val_loss: 0.2395 - val_acc: 0.9136 - lr: 0.0010
Epoch 4/100
238/238 [==============================] - ETA: 0s - loss: 0.2136 - acc: 0.9252- ETA: 0s - loss: 0.2142 - acc: 0
Epoch 00004: val_loss improved from 0.23918 to 0.23809, saving model to 64-basiccnn1d-word2vec-mort_hosp-best_model.hdf5
238/238 [==============================] - 18s 78ms/step - loss: 0.2136 - acc: 0.9252 - val_loss: 0.2381 - val_acc: 0.9140 - lr: 0.0010
Epoch 5/100
238/238 [==============================] - ETA: 0s - loss: 0.2038 - acc: 0.9282
Epoch 00005: val_loss did not improve from 0.23809
238/238 [==============================] - 20s 85ms/step - loss: 0.2038 - acc: 0.9282 - val_loss: 0.2397 - val_acc: 0.9131 - lr: 0.0010
Epoch 6/100
238/238 [==============================] - ETA: 

AUC:  0.8797889681910301 AUPRC:  0.5711206282953859 F1:  0.4779516358463727
0.8784050356859636 0.5663711488970622 0.9121435142594296 0.4246987951807229
Problem type:  mort_icu
__________________
x_train_lstm (15219, 24, 104) x_dev_lstm (2164, 24, 104)
x_train_ner (15219, 64, 100) x_dev_ner (2164, 64, 100)
(15219,)
(15219, 24, 104) (15219, 64, 100) 2
Epoch 1/100
238/238 [==============================] - ETA: 0s - loss: 0.2252 - acc: 0.9311
Epoch 00001: val_loss improved from inf to 0.18048, saving model to 64-basiccnn1d-word2vec-mort_icu-best_model.hdf5
238/238 [==============================] - 27s 95ms/step - loss: 0.2252 - acc: 0.9311 - val_loss: 0.1805 - val_acc: 0.9399 - lr: 0.0010
Epoch 2/100
238/238 [==============================] - ETA: 0s - loss: 0.1760 - acc: 0.9435
Epoch 00002: val_loss improved from 0.18048 to 0.17126, saving model to 64-basiccnn1d-word2vec-mort_icu-best_model.hdf5
238/238 [==============================] - 20s 82ms/step - loss: 0.1760 - acc: 0.9435 - val_

...

Epoch 9/100
238/238 [==============================] - ETA: 0s - loss: 0.1211 - acc: 0.9590
Epoch 00009: val_loss did not improve from 0.17388
238/238 [==============================] - 56s 236ms/step - loss: 0.1211 - acc: 0.9590 - val_loss: 0.1773 - val_acc: 0.9422 - lr: 4.0000e-05
AUC:  0.8869836290712736 AUPRC:  0.5088273455727309 F1:  0.4344086021505376
0.8860230659987203 0.5065196782228351 0.9392824287028518 0.41592920353982304
Iteration number:  3
Problem type:  mort_hosp
__________________
x_train_lstm (15219, 24, 104) x_dev_lstm (2164, 24, 104)
x_train_ner (15219, 64, 100) x_dev_ner (2164, 64, 100)
(15219,)
(15219, 24, 104) (15219, 64, 100) 2
Epoch 1/100
238/238 [==============================] - ETA: 0s - loss: 0.2848 - acc: 0.9027
Epoch 00001: val_loss improved from inf to 0.24764, saving model to 64-basiccnn1d-fasttext-mort_hosp-best_model.hdf5
238/238 [==============================] - 61s 233ms/step - loss: 0.2848 - acc: 0.9027 - val_loss: 0.2476 - val_acc: 0.9131 - lr: 0.

Epoch 4/100
238/238 [==============================] - ETA: 0s - loss: 0.1533 - acc: 0.9489
Epoch 00004: val_loss did not improve from 0.17253
238/238 [==============================] - 83s 348ms/step - loss: 0.1533 - acc: 0.9489 - val_loss: 0.1737 - val_acc: 0.9422 - lr: 0.0010
Epoch 5/100
238/238 [==============================] - ETA: 0s - loss: 0.1455 - acc: 0.9507
Epoch 00005: val_loss did not improve from 0.17253
238/238 [==============================] - 75s 314ms/step - loss: 0.1455 - acc: 0.9507 - val_loss: 0.1739 - val_acc: 0.9390 - lr: 0.0010
Epoch 6/100
238/238 [==============================] - ETA: 0s - loss: 0.1371 - acc: 0.9532
Epoch 00006: val_loss did not improve from 0.17253
238/238 [==============================] - 70s 295ms/step - loss: 0.1371 - acc: 0.9532 - val_loss: 0.1743 - val_acc: 0.9390 - lr: 2.0000e-04
Epoch 7/100
238/238 [==============================] - ETA: 0s - loss: 0.1347 - acc: 0.9545
Epoch 00007: val_loss did not improve from 0.17253
238/238 [====

Epoch 2/100
238/238 [==============================] - ETA: 0s - loss: 0.2386 - acc: 0.9175
Epoch 00002: val_loss improved from 0.24802 to 0.24097, saving model to 64-basiccnn1d-fasttext-mort_hosp-best_model.hdf5
238/238 [==============================] - 87s 364ms/step - loss: 0.2386 - acc: 0.9175 - val_loss: 0.2410 - val_acc: 0.9131 - lr: 0.0010
Epoch 3/100
238/238 [==============================] - ETA: 0s - loss: 0.2183 - acc: 0.9221
Epoch 00003: val_loss improved from 0.24097 to 0.23988, saving model to 64-basiccnn1d-fasttext-mort_hosp-best_model.hdf5
238/238 [==============================] - 72s 303ms/step - loss: 0.2183 - acc: 0.9221 - val_loss: 0.2399 - val_acc: 0.9127 - lr: 0.0010
Epoch 4/100
238/238 [==============================] - ETA: 0s - loss: 0.2049 - acc: 0.9271
Epoch 00004: val_loss did not improve from 0.23988
238/238 [==============================] - 64s 268ms/step - loss: 0.2049 - acc: 0.9271 - val_loss: 0.2422 - val_acc: 0.9127 - lr: 0.0010
Epoch 5/100
238/238 

x_train_lstm (15219, 24, 104) x_dev_lstm (2164, 24, 104)
x_train_ner (15219, 64, 100) x_dev_ner (2164, 64, 100)
(15219,)
(15219, 24, 104) (15219, 64, 100) 2
Epoch 1/100
238/238 [==============================] - ETA: 0s - loss: 0.2480 - acc: 0.9300
Epoch 00001: val_loss improved from inf to 0.18132, saving model to 64-basiccnn1d-fasttext-mort_icu-best_model.hdf5
238/238 [==============================] - 74s 280ms/step - loss: 0.2480 - acc: 0.9300 - val_loss: 0.1813 - val_acc: 0.9432 - lr: 0.0010
Epoch 2/100
238/238 [==============================] - ETA: 0s - loss: 0.1778 - acc: 0.9418
Epoch 00002: val_loss improved from 0.18132 to 0.17567, saving model to 64-basiccnn1d-fasttext-mort_icu-best_model.hdf5
238/238 [==============================] - 65s 274ms/step - loss: 0.1778 - acc: 0.9418 - val_loss: 0.1757 - val_acc: 0.9455 - lr: 0.0010
Epoch 3/100
238/238 [==============================] - ETA: 0s - loss: 0.1646 - acc: 0.9455
Epoch 00003: val_loss improved from 0.17567 to 0.17271, s

Epoch 4/100
238/238 [==============================] - ETA: 0s - loss: 0.1428 - acc: 0.9519
Epoch 00004: val_loss did not improve from 0.17650
238/238 [==============================] - 49s 206ms/step - loss: 0.1428 - acc: 0.9519 - val_loss: 0.1784 - val_acc: 0.9418 - lr: 0.0010
Epoch 5/100
238/238 [==============================] - ETA: 0s - loss: 0.1346 - acc: 0.9524 - ETA: 9s - loss: 0 - ETA:
Epoch 00005: val_loss did not improve from 0.17650
238/238 [==============================] - 48s 203ms/step - loss: 0.1346 - acc: 0.9524 - val_loss: 0.1785 - val_acc: 0.9422 - lr: 0.0010
Epoch 6/100
238/238 [==============================] - ETA: 0s - loss: 0.1222 - acc: 0.9581
Epoch 00006: val_loss did not improve from 0.17650
238/238 [==============================] - 50s 211ms/step - loss: 0.1222 - acc: 0.9581 - val_loss: 0.1796 - val_acc: 0.9418 - lr: 2.0000e-04
Epoch 7/100
238/238 [==============================] - ETA: 0s - loss: 0.1203 - acc: 0.9581
Epoch 00007: val_loss did not improve

embedding_types = ['word2vec', 'fasttext', 'concat']
embedding_dict = [ner_word2vec, ner_fasttext, ner_concat]

target_problems = [ 'mort_icu', 'los_3', 'los_7']#['mort_hosp']#,

num_epoch = 100
model_patience = 5
monitor_criteria = 'val_loss'
#monitor_criteria = 'val_acc'
batch_size = 64

filter_number = 32
ner_representation_limit = 64
activation_func = "relu"

sequence_model = "GRU"
sequence_hidden_unit = 256

maxiter = 11
for embed_dict, embed_name in zip(embedding_dict, embedding_types):    
    print ("Embedding: ", embed_name)
    print("=============================")
    
    temp_train_ner = dict((k, embed_dict[k]) for k in train_ids)
    tem_dev_ner = dict((k, embed_dict[k]) for k in dev_ids)
    temp_test_ner = dict((k, embed_dict[k]) for k in test_ids)

    x_train_dict = {}
    x_dev_dict = {}
    x_test_dict = {}

    x_train_dict = get_subvector_data(ner_representation_limit, embed_name, temp_train_ner)
    x_dev_dict = get_subvector_data(ner_representation_limit, embed_name, tem_dev_ner)
    x_test_dict = get_subvector_data(ner_representation_limit, embed_name, temp_test_ner)

    x_train_dict_sorted = collections.OrderedDict(sorted(x_train_dict.items()))
    x_dev_dict_sorted = collections.OrderedDict(sorted(x_dev_dict.items()))
    x_test_dict_sorted = collections.OrderedDict(sorted(x_test_dict.items()))
    
#     x_train_dict_sorted = sorted(x_train_dict.items())
#     x_dev_dict_sorted = sorted(x_dev_dict.items())
#     x_test_dict_sorted = sorted(x_test_dict.items())
    list_train_ner =[]
    list_test_ner =[]
    list_dev_ner =[]
    for k,v in x_train_dict_sorted.items():
            list_train_ner.append(v)
    for k,v in x_test_dict_sorted.items():
            list_test_ner.append(v)
    for k,v in x_dev_dict_sorted.items():
            list_dev_ner.append(v)
    
        
    
    x_train_ner = np.array(list_train_ner)
    x_dev_ner = np.array(list_dev_ner)
    x_test_ner = np.array(list_test_ner)
    
    print(x_train_ner.shape)
    

    #print('shape of x_train_ner{0} x_dev_ner{1} x_test_ner {2}'.format(x_train_ner.shape,x_dev_ner.shape,x_test_ner.shape))
    
        
    for iteration in range(1,maxiter):
        print ("Iteration number: ", iteration)
    
        for each_problem in target_problems:
            print ("Problem type: ", each_problem)
            print ("__________________")
            
            
            early_stopping_monitor = EarlyStopping(monitor=monitor_criteria, patience=model_patience)
            
            best_model_name = str(ner_representation_limit)+"-basiccnn1d-"+str(embed_name)+"-"+str(each_problem)+"-"+"best_model.hdf5"
            
            checkpoint = ModelCheckpoint(best_model_name, monitor=monitor_criteria, verbose=1,
                save_best_only=True, mode='min')
            
            reduce_lr = ReduceLROnPlateau(monitor=monitor_criteria, factor=0.2,
                              patience=2, min_lr=0.00001, epsilon=1e-4, mode='min')
            

            callbacks = [early_stopping_monitor, checkpoint, reduce_lr]
            
            #model = textCNN(sequence_model, sequence_hidden_unit, embed_name, ner_representation_limit)
            model = proposedmodel(sequence_model, sequence_hidden_unit, 
                               embed_name, ner_representation_limit,filter_number)
            print('x_train_lstm {0} x_dev_lstm {1}'.format(x_train_lstm.shape,x_dev_lstm.shape))
            print('x_train_ner {0} x_dev_ner {1}'.format(x_train_ner.shape,x_dev_ner.shape))
            print(y_train[each_problem].shape)
            aa = [x_train_lstm, x_train_ner]
            print(x_train_lstm.shape, x_train_ner.shape, len(aa))
            model.fit([x_train_lstm, x_train_ner], y_train[each_problem], epochs=num_epoch, verbose=1, 
                      validation_data=([x_dev_lstm, x_dev_ner], y_dev[each_problem]), callbacks=callbacks, batch_size=batch_size)
            
            
            probs, predictions = make_prediction_cnn(model, [x_test_lstm, x_test_ner])
            print_scores_cnn(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration, sequence_hidden_unit)
            
            model.load_weights(best_model_name)
                      
            probs, predictions = make_prediction_cnn(model, [x_test_lstm, x_test_ner])
            save_scores_cnn(predictions, probs, y_test[each_problem], embed_name, each_problem, iteration,
                            sequence_hidden_unit, sequence_model, type_of_ner)
            del model
            clear_session()
            gc.collect()
            